# Setup

In [ ]:
# Imports
import json
import glob
import pandas as pd
import numpy as np
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.nn.functional as F
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import string
import re
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader, Dataset

In [ ]:
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


# Dataset & GloVe

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# This is the train df, which will be broken up into a training set and a validation set
# df = pd.read_csv("/content/drive/Shareddrives/CIS530 Final Project/datasets/train.csv")
df = pd.read_csv("/content/drive/MyDrive/Copy of train.csv")

In [ ]:
# test_df = pd.read_csv("/content/drive/Shareddrives/CIS530 Final Project/datasets/test.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Copy of test.csv")

In [ ]:
df_en = df.loc[df['Language'] == 'en']
df_trun = df_en.sample(frac=1).groupby('Genre').head(1200)
X = df_trun['Lyrics']
Y = df_trun['Genre']
trainX, valX, trainY, valY = train_test_split(X, Y, test_size=0.15)

In [ ]:
trainY

165696          Folk
173142    Electronic
151359         Metal
272143          Jazz
190867          Folk
             ...    
288566    Electronic
158758          Folk
280924    Electronic
204            Metal
279512    Electronic
Name: Genre, Length: 10200, dtype: object

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
!wget -nc https://huggingface.co/stanfordnlp/glove/resolve/main/glove.6B.zip
!unzip glove.6B.zip
!ls -lat

File ‘glove.6B.zip’ already there; not retrieving.

Archive:  glove.6B.zip
replace glove.6B.100d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
  inflating: glove.6B.50d.txt        
total 3039152
drwxr-xr-x 1 root root       4096 Dec 21 08:44 .
drwx------ 5 root root       4096 Dec 21 08:07 drive
drwxr-xr-x 1 root root       4096 Dec 21 08:01 ..
drwxr-xr-x 1 root root       4096 Dec 19 14:20 sample_data
drwxr-xr-x 4 root root       4096 Dec 19 14:20 .config
-rw-r--r-- 1 root root  862182753 Aug 14  2021 glove.6B.zip
-rw-rw-r-- 1 root root  693434588 Dec 22  2015 glove.6B.200d.txt
-rw-rw-r-- 1 root root 1037965801 Dec 22  2015 glove.6B.300d.txt
-rw-rw-r-- 1 root root  171350515 Dec 22  2015 glove.6B.50d.txt
-rw-rw-r-- 1 root root  347117594 Dec 22  2015 glove.6B.100d.txt


In [ ]:
!head glove.6B.50d.txt

the 0.418 0.24968 -0.41242 0.1217 0.34527 -0.044457 -0.49688 -0.17862 -0.00066023 -0.6566 0.27843 -0.14767 -0.55677 0.14658 -0.0095095 0.011658 0.10204 -0.12792 -0.8443 -0.12181 -0.016801 -0.33279 -0.1552 -0.23131 -0.19181 -1.8823 -0.76746 0.099051 -0.42125 -0.19526 4.0071 -0.18594 -0.52287 -0.31681 0.00059213 0.0074449 0.17778 -0.15897 0.012041 -0.054223 -0.29871 -0.15749 -0.34758 -0.045637 -0.44251 0.18785 0.0027849 -0.18411 -0.11514 -0.78581
, 0.013441 0.23682 -0.16899 0.40951 0.63812 0.47709 -0.42852 -0.55641 -0.364 -0.23938 0.13001 -0.063734 -0.39575 -0.48162 0.23291 0.090201 -0.13324 0.078639 -0.41634 -0.15428 0.10068 0.48891 0.31226 -0.1252 -0.037512 -1.5179 0.12612 -0.02442 -0.042961 -0.28351 3.5416 -0.11956 -0.014533 -0.1499 0.21864 -0.33412 -0.13872 0.31806 0.70358 0.44858 -0.080262 0.63003 0.32111 -0.46765 0.22786 0.36034 -0.37818 -0.56657 0.044691 0.30392
. 0.15164 0.30177 -0.16763 0.17684 0.31719 0.33973 -0.43478 -0.31086 -0.44999 -0.29486 0.16608 0.11963 -0.41328 -0.42353

In [ ]:
glove_path = '/content/glove.6B.50d.txt'
glove_embeddings = {}
with open(glove_path, 'r', encoding='utf-8') as file:
    for line in file:
        values = line.split()
        word = values[0]
        vector = np.array(values[1:], dtype='float32')
        glove_embeddings[word] = vector

# Evaluation Functions

In [ ]:
all_genres = set(df['Genre'])

In [ ]:
# Returns percent accuracy, (precision, recall, f1)
def evaluate(pred_labels, labels, genres):
    if len(pred_labels) != len(labels):
        return "Label sizes are no the same"
    tracker = {}
    total_corr = 0
    for x in genres:
        tracker[x] = {'count': 0, 'tp': 0, 'fp': 0, 'fn': 0}
    all_tp = 0
    all_fp = 0
    all_fn = 0
    for pred, actual in zip(pred_labels, labels):
        tracker[actual]['count'] = tracker[actual]['count'] + 1
        if pred == actual:
            tracker[pred]['tp'] = tracker[pred]['tp'] + 1
            all_tp += 1
            total_corr += 1
        else:
            tracker[actual]['fn'] = tracker[actual]['fn'] + 1
            tracker[pred]['fp'] = tracker[pred]['fp'] + 1
            all_fp += 1
            all_fn += 1

    all_stats = {}
    for genre in tracker.keys():
        tp = tracker[genre]['tp']
        fp = tracker[genre]['fp']
        fn = tracker[genre]['fn']
        precision = tp/(tp+fp) if (tp+fp) > 0 else 0
        recall = tp/(tp+fn) if (tp+fn) > 0 else 0
        f1 = precision*recall/(precision+recall) if (precision+recall) > 0 else 0
        all_stats[genre] = {'count': tracker[genre]['count'], 'precision': precision, 'recall': recall, 'f1_score': f1}
    total_precision = all_tp/(all_tp+all_fp) if (all_tp+all_fp) > 0 else 0
    total_recall = all_tp/(all_tp+all_fn) if (all_tp+all_fn) > 0 else 0
    weighted_precision = sum([tracker[genre]['count']*all_stats[genre]['precision'] for genre in tracker.keys()])/len(pred_labels)
    weighted_recall = sum([tracker[genre]['count']*all_stats[genre]['recall'] for genre in tracker.keys()])/len(pred_labels)
    weighted_f1 = sum([tracker[genre]['count']*all_stats[genre]['f1_score'] for genre in tracker.keys()])/len(pred_labels)
    total_stats = {
        'macro_precision': total_precision,
        'macro_recall': total_recall,
        'macro_f1_score': total_precision*total_recall/(total_precision+total_recall)
            if (total_precision+total_recall) > 0 else 0,
        'weighted_precision': weighted_precision,
        'weighted_recall': weighted_recall,
        'weighted_f1_score': weighted_f1
    }
    return total_corr/len(pred_labels), total_stats, all_stats

In [ ]:
from tabulate import tabulate

def format_evaluation(stats):
    headers = list(all_genres)
    data = []
    for genre in headers:
        data.append([genre] + list(stats[2][genre].values()))
    res = tabulate(data, headers=['Genre', 'Count', 'Precision', 'Recall', 'F1_score'])
    res += '\n\nOverall\n'
    res += 'Accuracy: ' + str(stats[0]) + '\n' + \
        'Weighted Precision: ' + str(stats[1]['weighted_precision']) + '\n' + \
        'Weighted Recall: ' + str(stats[1]['weighted_recall']) + '\n' + \
        'Weighted F1 Score: ' + str(stats[1]['weighted_f1_score']) + '\n' + \
        'Macro Precision: ' + str(stats[1]['macro_precision']) + '\n' + \
        'Macro Recall: ' + str(stats[1]['macro_recall']) + '\n' + \
        'Macro F1 Score: ' + str(stats[1]['macro_f1_score'])
    return res

# Dataloading & Tokenizing

In [ ]:
stop_words = set(nltk.corpus.stopwords.words('english'))

In [ ]:
def preprocess_text(text):
  if type(text) == float:
    print(text)
  tokens = word_tokenize(text.lower())  # Tokenize and convert to lowercase
  tokens = [token for token in tokens if token.isalpha()]  # Remove non-alphabetic tokens
  tokens = [token for token in tokens if token not in stop_words]  # Remove stopwords
  return tokens

In [ ]:
df_en = df.loc[df['Language'] == 'en']
df_trun = df_en.sample(frac=1).groupby('Genre').head(1200)

In [ ]:
df_trun['Processed_Lyrics'] = df_trun['Lyrics'].apply(preprocess_text)

label_encoder = LabelEncoder()
df_trun['Genre_Encoded'] = label_encoder.fit_transform(df_trun['Genre'])

In [ ]:
test_df['Processed_Lyrics'] = test_df['Lyrics'].apply(preprocess_text)
test_df['Genre_Encoded'] = label_encoder.fit_transform(test_df['Genre'])

In [ ]:
train_df, val_df = train_test_split(df_trun, test_size=0.15, random_state=42)

In [ ]:
# Tokenization and creating word-to-index mapping
all_words = [word for tokens in train_df['Processed_Lyrics'] for word in tokens]
vocab = sorted(set(all_words))
word_to_idx = {word: idx for idx, word in enumerate(vocab)}

In [ ]:
# Create an embedding matrix
embedding_dim = 50  # Assuming you're using 50-dimensional GloVe embeddings
embedding_matrix = np.zeros((len(vocab), embedding_dim))
for word, idx in word_to_idx.items():
    if word in glove_embeddings:
        embedding_matrix[idx] = glove_embeddings[word]

In [ ]:
# Define a custom dataset class
class LyricsDataset(Dataset):
    def __init__(self, dataframe, word_to_idx, max_length):
        self.data = dataframe
        self.word_to_idx = word_to_idx
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        lyrics = self.data.iloc[idx]['Processed_Lyrics']
        encoded_lyrics = [self.word_to_idx[word] for word in lyrics if word in self.word_to_idx]
        padded_lyrics = encoded_lyrics[:self.max_length] + [0] * (self.max_length - len(encoded_lyrics))
        return torch.LongTensor(padded_lyrics), self.data.iloc[idx]['Genre_Encoded']

# Model

## Model Architecture

In [ ]:
# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.lstm(embedded)
        output = self.fc(output[:, -1, :])
        return output

In [ ]:
# Set hyperparameters
vocab_size = len(vocab)
hidden_dim = 128
output_dim = len(df['Genre'].unique())  # Number of unique genres in your dataset
max_length = 100  # Set your desired maximum sequence length

In [ ]:
# Create datasets and dataloaders
train_dataset = LyricsDataset(train_df, word_to_idx, max_length)
val_dataset = LyricsDataset(val_df, word_to_idx, max_length)

In [ ]:
test_dataset = LyricsDataset(test_df, word_to_idx, max_length)

In [ ]:
batch_size = 32  # Set your desired batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
# Initialize the model, loss function, and optimizer
model = LSTMModel(vocab_size, embedding_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## Training Loop

In [ ]:
# Training loop
num_epochs = 10  # Set your desired number of epochs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0

    for inputs, labels in train_loader:  # Iterate through the DataLoader
        inputs = inputs.to(device)  # Move inputs to device
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss / len(train_loader):.4f}")

Epoch [1/10], Loss: 2.2283
Epoch [2/10], Loss: 2.1766
Epoch [3/10], Loss: 2.1100
Epoch [4/10], Loss: 2.0192
Epoch [5/10], Loss: 1.9115
Epoch [6/10], Loss: 1.7887
Epoch [7/10], Loss: 1.6442
Epoch [8/10], Loss: 1.4830
Epoch [9/10], Loss: 1.3322
Epoch [10/10], Loss: 1.1870


## Validation Accuracy

In [ ]:
model.eval()
correct = 0
total = 0

val_predictions = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)

        # Ensure labels are tensors and move them to the device
        if not isinstance(labels, torch.Tensor):
            labels = torch.tensor(labels).to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        val_predictions.append(predicted)

        # Move predicted to the same device as labels for comparison
        predicted = predicted.to(device)

        total += labels.size(0)

        # Move labels to CPU for comparison
        correct += (predicted.cpu() == labels.cpu()).sum().item()

val_accuracy = correct / total * 100
print(f"Validation Accuracy: {val_accuracy:.2f}%")

Validation Accuracy: 23.61%


In [ ]:
val_predictions_flat = [int(item) for tensor in val_predictions for item in tensor.tolist()]

In [ ]:
reverse_encoder = LabelEncoder()
reverse_encoder.fit(list(all_genres))
val_genre_preds = label_encoder.inverse_transform(val_predictions_flat)

In [ ]:
val_genre_preds

array(['Hip-Hop', 'Rock', 'Pop', ..., 'Folk', 'Metal', 'Rock'],
      dtype=object)

In [ ]:
print(format_evaluation(evaluate(val_genre_preds, val_df['Genre'], all_genres)))

Genre         Count    Precision    Recall    F1_score
----------  -------  -----------  --------  ----------
Hip-Hop         164     0.552795  0.542683   0.273846
Rock            186     0.110345  0.172043   0.0672269
Indie           185     0.155039  0.108108   0.0636943
Pop             185     0.22      0.118919   0.077193
Metal           184     0.274611  0.288043   0.140584
R&B             167     0.221311  0.161677   0.0934256
Country         183     0.2       0.224044   0.10567
Jazz            190     0.348548  0.442105   0.194896
Electronic      189     0.183206  0.126984   0.075
Folk            167     0.144737  0.197605   0.0835443

Overall
Accuracy: 0.2361111111111111
Weighted Precision: 0.23870737810538922
Weighted Recall: 0.2361111111111111
Weighted F1 Score: 0.11635736238849749
Macro Precision: 0.2361111111111111
Macro Recall: 0.2361111111111111
Macro F1 Score: 0.11805555555555555


## Test Accuracy

In [ ]:
model.eval()
correct = 0
total = 0

test_predictions = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)

        # Ensure labels are tensors and move them to the device
        if not isinstance(labels, torch.Tensor):
            labels = torch.tensor(labels).to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        test_predictions.append(predicted)

        # Move predicted to the same device as labels for comparison
        predicted = predicted.to(device)

        total += labels.size(0)

        # Move labels to CPU for comparison
        correct += (predicted.cpu() == labels.cpu()).sum().item()

val_accuracy = correct / total * 100
print(f"Test Accuracy: {val_accuracy:.2f}%")

Test Accuracy: 23.62%


In [ ]:
test_predictions_flat = [int(item) for tensor in test_predictions for item in tensor.tolist()]
test_genre_preds = label_encoder.inverse_transform(test_predictions_flat)

In [ ]:
print(format_evaluation(evaluate(test_genre_preds, test_df['Genre'], all_genres)))

Genre         Count    Precision     Recall    F1_score
----------  -------  -----------  ---------  ----------
Hip-Hop         960    0.664642   0.569792    0.306786
Rock           1410    0.211915   0.176596    0.096325
Indie           510    0.0723684  0.0862745   0.039356
Pop            1110    0.211864   0.0900901   0.0632111
Metal           810    0.301994   0.392593    0.170692
R&B             510    0.124444   0.109804    0.0583333
Country         810    0.202105   0.237037    0.109091
Jazz            660    0.220794   0.286364    0.12467
Electronic      660    0.128165   0.122727    0.0626935
Folk            495    0.106987   0.19798     0.0694543

Overall
Accuracy: 0.23616887208569629
Weighted Precision: 0.247510020884755
Weighted Recall: 0.23616887208569629
Weighted F1 Score: 0.11783029097960623
Macro Precision: 0.23616887208569629
Macro Recall: 0.23616887208569629
Macro F1 Score: 0.11808443604284814
